In [138]:
import json
import random

In [375]:
#enter the file name that holds match data
file = "enter file name"
with open("../data/match_data/{}.json".format(file)) as json_data:
    d = json.load(json_data)

In [376]:
split_path_file = file.split("_")
match_day_round = split_path_file[-1].split(".")[0]

In [377]:
#dictionary that stores general information about the match
info = {
    "home":d["home"]["name"],
    "away":d["away"]["name"],
    "score_home": d["home"]["scores"]["fulltime"],
    "score_away": d["away"]["scores"]["fulltime"],
    "score": d["ftScore"],
    "venue": d["venueName"],
    "home_manager": d["home"]["managerName"],
    "away_manager":d["away"]["managerName"],
    "attendance" : d["attendance"]
    
}

In [378]:
#list with names of goalscores
score_names = []

for i in d["home"]["incidentEvents"]:
    #finding the goal scorers
    if i["type"]["displayName"] == "Goal":
        scorer_id = str(i["playerId"])
        score_names.append(d["playerIdNameDictionary"][scorer_id])
        
    else:
        pass

for i in d["away"]["incidentEvents"] :
    #finding the goal scorers
    if i["type"]["displayName"] == "Goal":
        scorer_id = str(i["playerId"])
        score_names.append(d["playerIdNameDictionary"][scorer_id])
        
    else:
        pass

In [379]:
#list when the goals were scored
score_time = []

#a list with the scoring team (1: home, 2: away)
score_team = []


for i in d["home"]["incidentEvents"]:
    if i["type"]["displayName"] == "Goal":
        #finding the scoring time
        score_time.append(i["minute"]+1)
        #home team scored
        score_team.append(1)
        
    else:
        pass

for i in d["away"]["incidentEvents"] :
    if i["type"]["displayName"] == "Goal":
        #finding the scoring time
        score_time.append(i["minute"]+1)
        #away team scored
        score_team.append(2)
    else:
        pass

In [380]:
#a list with names of players who assisted
score_assist = []

for i in d["home"]["incidentEvents"]:
    #finding the assistent
    try:
        if i["type"]["displayName"] == "Goal":
            assist_id = str(i["relatedPlayerId"])
            score_assist.append(d["playerIdNameDictionary"][assist_id])
    except:
        score_assist.append("no assist")
        
for i in d["away"]["incidentEvents"]:
    #finding the assistent
    try:
        if i["type"]["displayName"] == "Goal":
            assist_id = str(i["relatedPlayerId"])
            score_assist.append(d["playerIdNameDictionary"][assist_id])
    except:
        score_assist.append("no assist")



In [381]:
#sorting the arrays by the time the events occurred
try:
    #a_s: sorted score_time
    #b_s: sorted score_names
    #c_s: sorted score_assist
    #d_s: sorted score_team
    a_s, b_s, c_s, d_s = map(list, zip(*sorted(zip(score_time, score_names, score_assist,  score_team), reverse=False)))
except:
    pass

In [382]:
#finding the man of the match - if a player performance was outstanding he is mentioned in the title 
#if there is more than one goal scored

if info["score_home"] or info["score_home"] >= 1:
    
    man_of_match = []
    points = []
    num_goals = []
    num_assist = []

    for goal_scorer in set(b_s):
        if goal_scorer in set(c_s):
            man_of_match.append(goal_scorer)
            num_of_goals = b_s.count(goal_scorer)
            num_goals.append(num_of_goals)
            point_goal=num_of_goals * 5
            points.append(point_goal)
        else:
            man_of_match.append(goal_scorer)
            num_of_goals = b_s.count(goal_scorer)
            num_goals.append(num_of_goals)
            point_goal=num_of_goals * 5
            points.append(point_goal)
            num_assist.append(0)

    for assistant in set(c_s):
        if assistant in set(b_s):
            num_of_assists = c_s.count(assistant)
            num_assist.append(num_of_assists)
            point_goal=c_s.count(assistant) * 2.5
            point_s= point_goal + points[man_of_match.index(assistant)]
            points[man_of_match.index(assistant)] = point_s

        elif assistant not in man_of_match and assistant != "no assist":
            man_of_match.append(assistant)
            num_of_assists = c_s.count(assistant)
            num_assist.append(num_of_assists)
            point_goal=num_of_assists * 2.5
            points.append(point_goal)

        else:
            pass

else:
    pass

In [383]:
#loading the alternative description of a player
with open("../data/player_data/alt_player_info.json") as alt_data:
    alternative_names = json.load(alt_data)

In [384]:
#if there is more or one goal scored check if a player scored in a row to use alternative description
if info["score_home"] or info["score_away"] >= 1:
    #array that holds alternative names
    alt_sen = []
    for q,w in enumerate(b_s):
        for line in alternative_names:
            
            if line["full_name"] == w and q == 0:
                alt_sen.append(w)
            elif w == b_s[q-1] and line["full_name"] == w:
                #if all of the alternative namings were used, use surname
                if line["position"] in alt_sen:
                    alt_sen.append(line["surname"])
                #instead of player's surname use his position
                elif line["surname"] and line["age"] in alt_sen:
                    alt_sen.append(line["position"])
                #instead of player's full name use his age
                elif line["surname"] in alt_sen:
                    alt_sen.append(line["age"])

                else:
                    #instead of player's full name use his surname
                    alt_sen.append(line["surname"])
            #if the player is already on the score sheet but did not score in a row use his surname
            elif line["full_name"] == w and w in alt_sen:
                alt_sen.append(line["surname"])
            #use the full name if the player scored for the first time
            elif line["full_name"] == w:
                alt_sen.append(line["full_name"])


else:
    pass


In [385]:
def scorer_generator(string):

    if type(string) == int:
        return "the {} year old ".format(string)
    
    else:
        
        if string[0].isupper():
            return string

        else:
            return "the {}".format(string)

In [386]:
#function that creates headlines
def headline():
    #goalless draw
    if info["score_home"] == info["score_away"] and info["score_home"]==0:
        draw_no_goal = ["Goalless draw at {}".format(info["venue"]),
                "{}'s side fails to score in a {} draw".format(info["home_manager"], info["score"]),
               ]
        #random index to choose a headline from the group
        o = random.randrange(len(draw_no_goal))
        return draw_no_goal[o]
    
    else:
        #group of arrays that hold template sentences
        home_team_win = ["{}'s side wins at {} with {}:{}".format(info["home_manager"], info["venue"], info["score_home"], info["score_away"]),
                         "{}'s side looses at {} with {}:{}".format(info["away_manager"], info["venue"], info["score_away"], info["score_home"]),
                         "{} defeated {} at {}".format(info["home"], info["away"], info["venue"]),
                         "{} win over {} with {}".format(info["home"], info["away"], info["score"])]

        player_win_home = ["Magical performance by {} scoring {} goals".format(man_of_match[points.index(max(points))], num_goals[points.index(max(points))]),
                           "{} ensures {} sink {}".format(man_of_match[points.index(max(points))], info["home"], info["away"]),
                           "{} helps {} eclipse {}".format(man_of_match[points.index(max(points))], info["home"], info["away"]),
                           "{} inspires {} to victory over {}".format(man_of_match[points.index(max(points))], info["home"], info["away"]),
                           "{} seals points for {} against {}".format(man_of_match[points.index(max(points))], info["home"], info["away"])]

        away_team_win = ["{}'s side wins at {} with {}:{}".format(info["away_manager"], info["venue"], info["score_away"], info["score_home"]), 
                         "{} defeated {} at {}".format(info["away"], info["home"], info["venue"]),
                         "{} win over {} with {}:{}".format(info["away"], info["home"], info["score_away"], info["score_home"]),
                         "{}'s side looses at home with {}".format(info["home_manager"], info["score"]),
                        ]

        player_win_away = ["Magical performance by {}, scoring {} goals".format(man_of_match[points.index(max(points))], num_goals[points.index(max(points))]),
                                 "{} ensures {} sink {}".format(man_of_match[points.index(max(points))], info["away"], info["home"]),
                                 "{} helps {} eclipse {}".format(man_of_match[points.index(max(points))], info["away"], info["home"]),
                                 "{} inspires {} to victory over {}".format(man_of_match[points.index(max(points))], info["away"], info["home"]),
                                 "{} seals points for {} against {}".format(man_of_match[points.index(max(points))], info["away"], info["home"])]


        draw_goal = ["{} held to {} draw by {}".format(info["home"], info["venue"], info["away"]),
                     "{}'s side fails to win over {} in a {} draw".format(info["home_manager"], info["away"], info["score"]),
                     ]
        #random index to choose a headline from the group
        n = random.randrange(len(home_team_win))
        m = random.randrange(len(player_win_home))
        p = random.randrange(len(draw_goal))
        
        #draw
        if info["score_home"] == info["score_away"] and info["score_home"]>=1:
            return draw_goal[p]
        
        #home team wins   
        elif info["score_home"] > info["score_away"]:
            #mentioning the player in the headline if he reached the criteria
            if max(points) >= 15:
                return player_win_home[m]
            #returning other headlines
            else:
                return home_team_win[n]
        #away team wins
        else:
            #mentioning the player in the headline if he reached the criteria
            if max(points) >= 15:
                return player_win_away[m]
            #returning other headlines
            else:
                return away_team_win[n]


In [387]:
#function that return an ordinal number
def ordinal(integer):
    #integer to string to check which number was passed in

    num_string = str(integer)

    if num_string == "1" or num_string == "-1":
        return num_string+"st"
    elif num_string == "2" or num_string == "-2":
        return num_string+"nd"
    elif num_string == "3" or num_string == "-3":
        return num_string+"rd"

    elif num_string[-1] == "1" and num_string[-2] != "1":
        return num_string+"st"
    elif num_string[-1] == "2" and num_string[-2] != "1":
        return num_string+"nd"
    elif num_string[-1] == "3" and num_string[-2] != "1":
        return num_string+"rd"

    else:
        return num_string+"th"

In [388]:
#function that generates the main body of the article
def goals():
    #score counter
    score_home = 0

    score_away = 0
    #string that holds the article
    ret = ""

    for i, e in enumerate(d_s):
        
        #groups of arrays that hold sentence templates
        first_scored_assist_home = ["{} scored the opener in the {} minute, with assistance from {}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), c_s[i]),
                                    "In the {} minute {} gave {} lead over {} with an assist by {} . ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["home"], info["away"], c_s[i]),
                                    "It was in the {} minute of the match when {} scored the opener at {}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["venue"]),
                                    "Home fans were delighted when {} scored the first goal of the match. He netted the ball in the {} minute with assistance from {}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), c_s[i]),
                                    "{} fans saw the first goal in the {} minute. {} scored after an assist by {}. ".format(info["attendance"], ordinal(a_s[i]), scorer_generator(alt_sen[i]), c_s[i])]
        
        
        first_scored_assist_away = ["{} scored the opener in the {} minute, with assistance from {}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), c_s[i]),
                                    "In the {} minute {} gave {} lead over {} with an assist by {}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["away"], info["home"], c_s[i]),
                                    "It was in the {} minute of the match when {} scored the opener at {}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["venue"]),
                                    "Home fans were not delighted when {} scored the first goal in the match. He netted the ball in the {} minute with assistance from {}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), c_s[i]),
                                    "{} fans saw the first goal in the {} minute. {} scored after an assist by {}.".format(info["attendance"], ordinal(a_s[i]), scorer_generator(alt_sen[i]), c_s[i])]

        first_scored_home = ["{} scored the opener in the {} minute. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i])),
                     "In the {} minute {} gave {} a lead over {}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["home"], info["away"]),
                     "It was in the {} minute when {} scored the opener at {}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["venue"]),
                     "Home fans were delighted when {} scored the first goal in the match. He netted the ball in the {} minute. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i])),
                     "{} fans saw the first goal in the {} minute, scored by {}. ".format(info["attendance"], ordinal(a_s[i]), scorer_generator(alt_sen[i]))]

        first_scored_away = ["{} scored the opener in the {} minute. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i])),
                             "In the {} minute {} gave {} a lead over {}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["away"], info["home"]),
                             "It was in the {} minute when {} scored the opener at {}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["venue"]),
                             "Home fans were gutted when {} scored the first goal in the match. He netted the ball in the {} minute. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i])),
                             "{} fans saw the first goal in {} minute, scored by {}. ".format(info["attendance"], ordinal(a_s[i]), scorer_generator(alt_sen[i]))
                            ]

        scored_assist_advance_home = ["After an assist by {} {} shot {} in advance, leading {}:{} in the {} minute. ".format(c_s[i], scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away, ordinal(a_s[i])),
                                      "{} scored in {} minute with the assist of {} for {}:{}".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), c_s[i], score_home+1, score_away),
                                      "Home fans were delighted when {} assisted {} who increased the lead to {}:{} at the {} minute mark. ".format(c_s[i], scorer_generator(alt_sen[i]), score_home+1, score_away, a_s[i]),
                                      "In the {} minute, {} and {} lead to the delight of home fans the team from {} into an advantage with {}:{}. ".format(ordinal(a_s[i]),c_s[i], scorer_generator(alt_sen[i]), info["venue"], score_home+1, score_away),
                                      "It was in the {} when {} assisted {} who extended the lead for {} to {}:{}. ".format(ordinal(a_s[i]), c_s[i], scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away),
                                      "{} assisted {} who scored on the joy of home fans in {} minute. {} {} {} {}. ".format(c_s[i], scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["home"], score_home+1, info["away"], score_away)]

        
        

        scored_assist_advance_away = ["After an assist by {} {} shot {} in advance, leading {}:{} in the {} minute. ".format(c_s[i], scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1, ordinal(a_s[i])),
                                      "{} scored in {} minute with the assist of {} for {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), c_s[i], score_home, score_away+1),
                                      "{} home fans at {} were shocked when {} assisted {} who increased the lead for {} to {}:{} at the {} minute mark. ".format(info["attendance"], info["venue"], c_s[i], scorer_generator(alt_sen[i]),info["away"], score_home, score_away+1, a_s[i]),
                                      "Home fans were dismayed when {} and {} lead {} into an advantage in the {} minute with {}:{}. ".format(c_s[i], scorer_generator(alt_sen[i]), info["away"], ordinal(a_s[i]), score_home, score_away+1),
                                      "It was in the {} minute when {} assisted {} who extended the lead for {} to {}:{}. ".format(ordinal(a_s[i]), c_s[i], scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1),
                                      "{} assisted {} who scored on the horror of home fans in the {} minute. {} {} {} {}. ".format(c_s[i], scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["home"], score_home, info["away"], score_away+1) 
                                        ]

        
        scored_advance_home = ["In the {} minute {} shot {} in advance, leading {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away),
                                "{} scored in {} minute, putting {} {}:{} in the lead. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["home"], score_home+1, score_away),
                                "{} led the home team into a {}:{} advance in the {} minute. ".format(scorer_generator(alt_sen[i]), score_home+1, score_away, ordinal(a_s[i])),
                                "{} led {} into {}:{} advance in the {} minute. ".format(scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away, ordinal(a_s[i])),
                                "It was in the {} minute when {} extended the lead for {} to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away),
                                "{} scored on the satisfaction of home fans in {} minute. {} was {}:{} ahead of {}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["home"], score_home+1, score_away, info["away"]) 
                                ]

        scored_advance_away = ["In the {} minute {} shot {} in advance, leading {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1),
                                "{} scored in {} minute, putting {} {}:{} in the lead. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["away"], score_home, score_away+1),
                                "{} led the away team into a {}:{} advance in the {} minute. ".format(scorer_generator(alt_sen[i]), score_home, score_away+1, ordinal(a_s[i])),
                                "{} led {} into {}:{} advance in the {} minute. ".format(scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1, ordinal(a_s[i])),
                                "It was in the {} minute when {} extended the lead for {} to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1),
                                "Home fans were horrified when {} scored in the {} minute. {} was {}:{} ahead of {}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["away"], score_home, score_away+1, info["home"]) 
                                ]

        
        scored_assist_reduce_home = ["It was in the {} minute when {} with an assist from {} reduced the score to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), c_s[i], score_home+1, score_away),
                                    "{} fans got a bit hopeful when {} assisted {}. It was the {} minute and the score was {}:{}. ".format(info["home"], c_s[i], scorer_generator(alt_sen[i]), ordinal(a_s[i]), score_home+1, score_away),
                                    "{} scored after assistance from {} in the {} minute reducing the score to {}:{}. ".format(scorer_generator(alt_sen[i]), c_s[i], ordinal(a_s[i]), score_home+1, score_away),
                                    "After an assist by {} {} scored for {}, lowering the gap to {}:{}. ".format(c_s[i], scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away),
                                    "After assistance from {} {} netted the ball for {}, trailing {}:{} behind {}. ".format(c_s[i], scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away, info["away"]),
                                    ]

        scored_assist_reduce_away = ["It was in the {} minute when {} with an assist from {} reduced the score to {}:{}. ".format(a_s[i], scorer_generator(alt_sen[i]), c_s[i], score_home, score_away+1),
                                    "{} fans got a bit hopeful when {} assisted {}. It was the {} minute and the score was {}:{}. ".format(info["away"], c_s[i], scorer_generator(alt_sen[i]), a_s[i], score_home, score_away+1),
                                    "{} scored after assistance from {} in the {} minute reducing the score to {}:{}. ".format(scorer_generator(alt_sen[i]), c_s[i], a_s[i], score_home, score_away+1),
                                    "After an assist by {} {} scored for {}, lowering the gap to {}:{}. ".format(c_s[i], scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1),
                                    "After assistance from {} {} netted the ball for {}, trailing {}:{} behind {}. ".format(c_s[i], scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1, info["home"]),
                                    ]
        scored_reduce_home = ["It was in the {} minute when {} reduced the gap to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), score_home+1, score_away),
                              "{} fans got a bit hopeful when {} scored. It was the {} minute and the score was {}:{}. ".format(info["home"], scorer_generator(alt_sen[i]), ordinal(a_s[i]), score_home+1, score_away),
                              "It was in the {} minute when {} reduced the gap to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), score_home+1, score_away),
                              "{} scored in the {} minute reducing the score to {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), score_home+1, score_away),
                              "In the {} mminute {} scored for {}, lowering the gap to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away),
                              "{} netted the ball for {} in the {} minute, trailing {}:{} behind {}. ".format(scorer_generator(alt_sen[i]), info["home"], ordinal(a_s[i]), score_home+1, score_away, info["away"]),
                              ]

        scored_reduce_away = ["It was in the {} minute when {} reduced the gap to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), score_home, score_away+1),
                              "{} fans got a bit hopeful when {} scored. It was the {} minute and the score was {}:{}. ".format(info["away"], scorer_generator(alt_sen[i]), ordinal(a_s[i]), score_home, score_away+1),
                              "It was in the {} minute when {} reduced the gap to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), score_home, score_away+1),
                              "{} scored in the {} minute reducing the gap to {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), score_home, score_away+1),
                              "In the {} mminute {} scored for {}, lowering the gap to {}:{}. ".format(ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1),
                              "{} netted the ball for {} in the {} minute, trailing {}:{} behind {}. ".format(scorer_generator(alt_sen[i]), info["away"], ordinal(a_s[i]), score_home, score_away+1, info["home"]),
                              ]

        
        first_last_home_assist = ["The match at {} was decided in the {} minute. The only goal in the match was scored by {} after an assist from {}. Although seeing only one goal, {} home fans were delighted as {} secured 3 points. {} {} {}:{}. ".format(info["venue"], ordinal(a_s[i]), scorer_generator(alt_sen[i]), c_s[i], info["attendance"], info["home"], info["home"], info["away"], info["score_home"], info["score_away"]),
                                 ]
        first_last_away_assist = ["The match at {} was decided in the {} minute. The only goal in the match was scored by {} after an assist from {}. Although seeing only one goal, away fans were delighted as {} secured 3 points. {} {} {}:{}. ".format(info["venue"], ordinal(a_s[i]), scorer_generator(alt_sen[i]), c_s[i], info["away"], info["home"], info["away"], info["score_home"], info["score_away"]),
                                  ]
        
        first_last_home = ["The match at {} was decided in the {} minute. The only goal in the match was scored by {}. Although seeing only one goal, {} home fans were delighted as {} secured 3 points. {} {} {}:{}. ".format(info["venue"], ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["attendance"], info["home"], info["home"], info["away"], info["score_home"], info["score_away"])]
        
        first_last_away = ["The match at {} was decided in the {} minute. The only goal in the match was scored by {}. Although seeing only one goal, {} away fans were delighted as {} secured 3 points. {} {} {}:{}. ".format(info["venue"], ordinal(a_s[i]), scorer_generator(alt_sen[i]), info["attendance"], info["away"], info["home"], info["away"], info["score_home"], info["score_away"]),]
        
        last_assist_home = ["{} settled the score of the match after assistance from {} in {} minute. {} {} {} {}. ".format(scorer_generator(alt_sen[i]), c_s[i], ordinal(a_s[i]), info["home"], score_home+1, info["away"], score_away),
                            "{} scored the last goal of the match in {} minute who was assisted by {}. {} {} {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), c_s[i], info["home"], info["away"], score_home+1, score_away),]


        last_assist_away = ["{} settled the score of the match after an assist by {} in {} minute. {} {} {} {}. ".format(scorer_generator(alt_sen[i]), c_s[i], ordinal(a_s[i]), info["home"], score_home, info["away"], score_away+1),
                            "{} scored the last goal of the match in {} minute who was assisted by {}. {} {} {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), c_s[i], info["home"], info["away"], score_home, score_away+1)]


        last_home = ["{} settled the score of the match in {} minute. {} {} {} {}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["home"], score_home+1, info["away"], score_away),
                    "{} scored the last goal of the match in {} minute. {} {} {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["home"], info["away"], score_home+1, score_away)]

        last_away = ["{} settled the score of the match in {} minute. {} {} {} {}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["home"], score_home, info["away"], score_away+1),
                    "{} scored the last goal of the match in {} minute. {} {} {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), info["home"], info["away"], score_home, score_away+1)]

        draw_assist_home = ["Home fans were delighted when {} assisted {} and gave hope to {}. Score was drew to {}:{}. ".format(c_s[i], scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away), 
                            "{} assisted {} who netted the ball to the delight of home fans and levelled the score to {}:{}. ".format(c_s[i], scorer_generator(alt_sen[i]), score_home+1, score_away)
                            ]

        draw_assist_away = ["Home fans were dissapointed when {} assisted {} and returned {} back into the match. Score was drew to {}:{}. ".format(c_s[i], scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1),
                            "{} assisted {} who netted the ball to the horror of home fans and levelled the score to {}:{}. ".format(c_s[i], scorer_generator(alt_sen[i]), score_home, score_away+1)
                            ]

        draw_home = ["Home fans were delighted when {} scored and gave hope to {}. Score was drew to {}:{}. ".format(scorer_generator(alt_sen[i]), info["home"], score_home+1, score_away), 
                    "{} netted the ball in the {} minute to the delight of home fans and levelled the score to {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), score_home+1, score_away),
                    ]

        draw_away = ["Home fans were dissapointed when {} scored and gave hope to {}. Score was drew to {}:{}. ".format(scorer_generator(alt_sen[i]), info["away"], score_home, score_away+1), 
                    "{} netted the ball in the {} minute to the delight of away fans and levelled the score to {}:{}. ".format(scorer_generator(alt_sen[i]), ordinal(a_s[i]), score_home, score_away+1),
                    ]
        #random index to choose a headline from the group
        m = random.randrange(len(first_last_home_assist))
        n = random.randrange(len(first_scored_assist_home))
        o = random.randrange(len(first_scored_home))
        p = random.randrange(len(scored_assist_reduce_home))
        r = random.randrange(len(scored_reduce_home))
        s = random.randrange(len(scored_assist_advance_home))
        t = random.randrange(len(scored_advance_home))
        u = random.randrange(len(draw_home))
        w = random.randrange(len(last_assist_home))
        x = random.randrange(len(first_last_home))
        y = random.randrange(len(last_home))
        z = random.randrange(len(draw_assist_home))
        
        #home team scores
        if e == 1:
            score_home += 1
            
            #first and last 
            if len(d_s) == 1 and c_s[i] != "no assist":
                return first_last_home_assist[m]
            
            elif len(d_s) == 1 and c_s[i] == "no assist":
                return first_last_home[x]
            
            #first goal in match
            elif i == 0 and c_s[i] != "no assist":
                ret += first_scored_assist_home[n]
                
            elif i == 0 and c_s[i] == "no assist":
                ret += first_scored_home[o]
                
            #last goal
            elif d_s[-1] == e and (len(d_s) - 1) == i and c_s[i] != "no assist":
                ret += last_assist_home[w]
                
            elif d_s[-1] == e and (len(d_s) - 1) == i and c_s[i] == "no assist":
                ret += last_home[y]
                
            #reduce the score
            elif (score_home and score_away - score_home >= 1) and c_s[i] != "no assist":
                ret += scored_assist_reduce_home[p]
    
            elif (score_home and score_away - score_home >= 1) and c_s[i] == "no assist":
                ret += scored_reduce_home[r]

#           advance the lead
            elif (score_home - score_away >= 1) and c_s[i] != "no assist":
                ret += scored_assist_advance_home[s]
            
            elif (score_home - score_away >= 1) and c_s[i] == "no assist":
                ret += scored_advance_home[t]
                
            #draw score
            
            elif (score_home - score_away == 0) and c_s[i] != "no assist":
                ret += draw_assist_home[z]

            else:
                ret += draw_home[u]
        else:
            
            score_away += 1
            
            #first and last 
            if len(d_s) == 1 and c_s[i] != "no assist":
                return first_last_away_assist[m]
            
            elif len(d_s) == 1 and c_s[i] == "no assist":
                return first_last_home[x]
            
            #first goal in match
            
            elif i == 0 and c_s[i] != "no assist":
                ret += first_scored_assist_away[n]
    
            elif i == 0 and c_s[i] == "no assist":
                ret += first_scored_away[o]
            
            #last goal
            elif d_s[-1] == e and (len(d_s) - 1) == i and c_s[i] != "no assist":
                ret += last_assist_away[w]
                
            elif d_s[-1] == e and (len(d_s) - 1) == i and c_s[i] == "no assist":
                ret += last_away[y]
            
            #reduce the score
            elif (score_away and score_home - score_away >= 1) and c_s[i] != "no assist":
                ret += scored_assist_reduce_away[p]
                
            elif (score_away and score_home - score_away >= 1) and c_s[i] == "no assist":
                ret += scored_reduce_away[r]
                
            #advance the lead
            elif (score_away - score_home >= 1) and c_s[i] != "no assist":
                ret += scored_assist_advance_away[s]
            
            elif (score_away - score_home >= 1) and c_s[i] == "no assist":
                ret += scored_advance_away[t]
                
             #draw score
            elif (score_home - score_away == 0) and c_s[i] != "no assist":
                ret += draw_assist_away[z]

            else:
                ret += draw_away[u]
                
    return(ret)



In [389]:
#content function returns the result of the goals function or if the match ended in a goalless draw, 
#it returns a predefined template.
def content():
    
    #draw no goals
    if info["score_home"] == 0 and info["score_away"] == 0:
    
        draw_no_goal = """The match at {} was nothing for a goal hungry crowd. Probably most of the {} fans left the stadium disappointed as neither of the teams were able to put their players on the scoresheet. {} {} {}:{}.""".format(info["venue"], info["attendance"], info["home"], info["away"], 0, 0)
        return draw_no_goal
    
    #draw goal
    elif info["score_home"] == info["score_away"] and info["score_home"]>=1:
        return goals()
    
    #first goal also last goal
    elif (info["score_home"] > info["score_away"] or info["score_away"] > info["score_home"]) and len(a_s) == 1:
        return goals()
    
    else:
        return goals()

In [390]:
#function that return singular or plural form to be used when describing standing in league table
def plural(num, string):
    
    if num == 1:
        return "{} {}".format(num, string)
    elif num == 0:
        return "{} {}".format(num, string)+"s"
    else:
        return "{} {}".format(num, string)+"s"

In [391]:
#function that returns the conclusion of the article - about the standings
def table():
       
    #current matchday
    with open("../data/matchday_table/matchday{}.json".format(match_day_round)) as json_table:
        t = json.load(json_table)
    #previous matchday if it is not the 1st
    if int(match_day_round) > 1:
        with open("../data/matchday_table/matchday{}.json".format(int(match_day_round)-1)) as prev_round:
            p = json.load(prev_round)
    #dictionary that stores information about the standings - to be used in templates
    data_table = {
        "home_pos": [],
        "away_pos": [],
        "home_pts": [],
        "away_pts": [],
        "home_prev_pos": [],
        "away_prev_pos": [],
        "home_prev_pts": [],
        "away_prev_pts": [],    
    }
    
    #looping through the tables and storing the information in the dictionary
    for i in t:
        if i["club_name"] == info["home"]:
            
            data_table["home_pos"].append(int(i["position"]))
            data_table["home_pts"].append(int(i["points"]))            
        
        elif i["club_name"] == info["away"]:
        
            data_table["away_pos"].append(int(i["position"]))
            data_table["away_pts"].append(int(i["points"]))
    if int(match_day_round) > 1:        
        for j in p:
            if j["club_name"] == info["home"]:

                data_table["home_prev_pos"].append(int(j["position"]))
                data_table["home_prev_pts"].append(int(j["points"]))


            elif j["club_name"] == info["away"]:

                data_table["away_prev_pos"].append(int(j["position"]))            
                data_table["away_prev_pts"].append(int(j["points"]))
            
    #Created specially for the first matchday
    if match_day_round == "1":
        
        #home win
        if  info["score_home"] >  info["score_away"]:
            return "{} fans saw their team collecting first three points in the season. {} are returning home empty handed".format(info["home"], info["away"])

        #away win
        elif  info["score_away"] >  info["score_home"]:
            return "{} can be delighted with openning the season away at {}. They are returning home with three points in their bag".format(info["away"], info["home"])

        #draw
        else:
            return "The season opener for {} and {} ended with a draw. The teams go into the second round of Premier League with 1 point in their account.".format(info["home"], info["away"])

    else:
        #home wins
        if  info["score_home"] >  info["score_away"]:

            #home advance, away drop
            if  data_table["home_pos"] < data_table["home_prev_pos"] and data_table["away_pos"] > data_table["away_prev_pos"]:
                return "{} advanced with this win from {} place to {} place with {}. {} on the other hand have {} and dropped from {} to {} place.".format(info["home"], ordinal(data_table["home_prev_pos"][0]), ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], plural(data_table["away_pts"][0], "point"), ordinal(data_table["away_prev_pos"][0]), ordinal(data_table["away_pos"][0]))

            #home advance, away stays
            elif data_table["home_pos"] < data_table["home_prev_pos"] and data_table["away_pos"] == data_table["away_prev_pos"]:
                return "{} advanced with this win from {} place to {} place with {}. The defeat didn't cause any harm to {} as they hold the {} position with {} from last round.".format(info["home"], ordinal(data_table["home_prev_pos"][0]), ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], ordinal(data_table["away_prev_pos"][0]), plural(data_table["away_pts"][0], "point"))

            #home stays, away drops
            elif data_table["home_pos"] == data_table["home_prev_pos"] and data_table["away_pos"] > data_table["away_prev_pos"]:
                return "{} kept the {} place from the last round with the win, having {}. {} dropped with this defeat to {} place with {}.".format(info["home"], ordinal(data_table["home_prev_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))

            #home stays, away stays
            else:
                return "In the terms of the league table nothing changed for the teams. {} is still on the {} place with {}. {} can be found on {} place, having {} on their account.".format(info["home"], ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))

        #away win
        elif info["score_away"] >  info["score_home"]:
            #away advance, home drop
            if  data_table["away_pos"] < data_table["away_prev_pos"] and data_table["home_pos"] > data_table["home_prev_pos"]:
                return "{} advanced with this win from {} place to {} place with {}. {} on the other hand have {} and dropped from {} to {} place.".format(info["away"], ordinal(data_table["away_prev_pos"][0]), ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"), info["home"], plural(data_table["home_pts"][0], "point"), ordinal(data_table["home_prev_pos"][0]), ordinal(data_table["home_pos"][0]))        
            #away advance, home stays
            elif data_table["away_pos"] < data_table["away_prev_pos"] and data_table["home_pos"] == data_table["home_prev_pos"]:
                return "{} advanced with this win from {} place to {} place with {}. The defeat didn't cause any harm to {} as they hold the {} position with {} from last round.".format(info["away"], ordinal(data_table["away_prev_pos"][0]), ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"), info["home"], ordinal(data_table["home_prev_pos"][0]), plural(data_table["home_pts"][0], "point"))

            #away stays, home drops
            elif data_table["away_pos"] == data_table["away_prev_pos"] and data_table["home_pos"] > data_table["home_prev_pos"]:
                return "{} kept the {} place from the last round with the win, having {}. {} dropped with this defeat to {} place with {}.".format(info["away"], ordinal(data_table["away_prev_pos"][0]), plural(data_table["away_pts"][0], "point"), info["home"], ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"))

            #away stays, home stays
            else:
                return "In the terms of the league table nothing changed for the teams. {} is still on the {} place with {}. {} can be found on {} place, having {} on their account.".format(info["away"], ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"), info["home"], ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"))

        #draw
        else:
            #both advance
            if data_table["home_pos"] < data_table["home_prev_pos"] and data_table["away_pos"] < data_table["away_prev_pos"]:
                return "Both teams were happy with the draw as they advanced on the table. {} are now {} with {}. {} can be found on {} with {}.".format(info["home"], ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))

            #both drop
            elif  data_table["home_pos"] > data_table["home_prev_pos"] and data_table["away_pos"] > data_table["away_prev_pos"]:
                return "The teams cannot be happy with the draw as both dropped on the table. {} dropped from {} to {} place, currently with {} on their account. {} lost {} and are now {} with {}.".format(info["home"], ordinal(data_table["home_prev_pos"][0]), ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], plural(data_table["away_pos"][0]-data_table["away_prev_pos"][0], "place"), ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))

            #both stay
            elif data_table["home_pos"] == data_table["home_prev_pos"] and data_table["away_pos"] == data_table["away_prev_pos"]:
                return "The draw didn't change anything for teams as both kept their positions from previous round. {} are {} with {} and {} can be found on {} place with {}.".format(info["home"], ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))

        #         #home advances, away drops
            elif data_table["home_pos"] < data_table["home_prev_pos"] and data_table["away_pos"] > data_table["away_prev_pos"]:
                return "{} cannot be disappointed with the draw as they gained {} and are now {}. {} can be found on {} place with {}.".format(info["home"], plural((data_table["home_prev_pos"][0]-data_table["home_pos"][0]), "place"), ordinal(data_table["home_pos"][0]), info["away"], ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))

            #home advances, away stays
            elif data_table["home_pos"] < data_table["home_prev_pos"] and data_table["away_pos"] == data_table["away_prev_pos"]:
                return "The draw brought more joy to {} as they advanced from {} to {} place. Currently they have {}. {} kept the {} position with {}.".format(info["home"], ordinal(data_table["home_prev_pos"][0]), ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))

            #home drops, away advances
            elif data_table["home_pos"] > data_table["home_prev_pos"] and data_table["away_pos"] < data_table["away_prev_pos"]:
                return "Home fans can not be delighted with the draw as they saw {} dropping from {} to {} place, having {} in their account. {} fans saw their team advancing from {} to {} place, currently holding {}.".format(info["home"], ordinal(data_table["home_prev_pos"][0]), ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], ordinal(data_table["away_prev_pos"][0]), ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))

            #home drops, away stays
            elif data_table["home_pos"] > data_table["home_prev_pos"] and data_table["away_pos"] == data_table["away_prev_pos"]:
                return "Home fans can not be delighted with the draw as they saw {} dropping from {} to {} place, having {} in their account. {} kept with {} their {} position.".format(info["home"], ordinal(data_table["home_prev_pos"][0]), ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], plural(data_table["away_pts"][0], "point"), ordinal(data_table["away_pos"][0]))        

            #home stays, away advances
            elif data_table["home_pos"] == data_table["home_prev_pos"] and data_table["away_pos"] < data_table["away_prev_pos"]:
                return "{} are returning home with one point which helped them to advance from {} to {} place, holding {}. {} kept with {} their {} position.".format(info["away"], ordinal(data_table["away_prev_pos"][0]), ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"), info["home"], plural(data_table["home_pts"][0], "point"), ordinal(data_table["home_pos"][0]))

            #home stays, away drops
            elif data_table["home_pos"] == data_table["home_prev_pos"] and data_table["away_pos"] > data_table["away_prev_pos"]:
                return "Nothing has changed for {} as they kept their {} position with {}. Away fans won't be pleased when returning home as {} lost {}, dropping to {} position with {}.".format(info["home"], ordinal(data_table["home_pos"][0]), plural(data_table["home_pts"][0], "point"), info["away"], plural(data_table["away_pos"][0] - data_table["away_prev_pos"][0], "place"), ordinal(data_table["away_pos"][0]), plural(data_table["away_pts"][0], "point"))     



In [392]:
#function assembles the article and writes it to a text file
def article():
    
    news_article =  headline() + 2*"\n" + content() + 2*"\n" + table() 
    print(news_article)
    with open("articles_stage_3/output_{}.txt".format(file), "w") as text_file:
        text_file.write(news_article)

article()

Sean Dyche's side looses at Emirates Stadium with 1:2

59955 fans saw the first goal in the 59th minute. Shkodran Mustafi scored after an assist by Mesut Özil. Andre Gray netted the ball in the 93rd minute to the delight of away fans and levelled the score to 1:1. Alexis Sánchez scored the last goal of the match in 98th minute. Arsenal Burnley 2:1. 

Arsenal advanced with this win from 4th place to 2nd place with 47 points. Burnley on the other hand have 26 points and dropped from 10th to 13th place.
